# Creating neural net components based on node and 2d features

General model architecture:
- Accept positions, node features, edges and edge features (edge graph precomputed)
- node features and edge features embedded into node and edge attributes (including distance)
- node and edge hidden state updated with MLPs
- Determinine l features due to eq. bond length, planar, and chirality constraints
- Combine all into SO3 embeddings with scalars, l1 features, and map edge features onto all angular momenta but aligning the SO3 embeddings with the edge and then rotating after m=0 component has been computed
- Attention layers on SO3 embeddings a la Equiformer, with a slight tweak: originally, only distance and optionally atom identies was used to attenuate attention weight. Here we use node and edge ATTRIBUTES concatenated along with distance.
- Heads:
  - Node classifier: Extract l=0 features and give to MLP
  - Denoiser: One more SO3MLP laper and extract l=1 features, mean over the channels and convert from spherical harmonics to cartesian coordinates, outputting a vector
  - Global classifier: TBD need a pooling mechanism


## Backbone architecture

0. Inputs are: 
  - atom and bond features $f_a, [N, ?]$
  - positions $r, [N, 3]$
  - edge features $f_e, [E, ?]$
  - edge distances $R, [E,1]$

1. Embed atom and bond features to node and edge attributes:
  - $f_a' = Lin(Concat_{f_a^i}^{f_a}(Emb(f_a^i)))$
  - $f_e' = Lin(Concat_{f_e^i}^{f_e}(Emb(f_e^i)))$
2. Radial basis expansion of edge distances:
  - $R' = RadialBasis(R)$
3. Compute l1 vectors based on topology:
  - $f_a^{l1} = get\_gradients(topology, r), [N,3,3]$
  - I don't think conversion to spherical harmonics is needed here as they will go into a linear layer
4. Generate SO3 initial by:
  - Node attributes $f_a^{l1}$ assigned to l=0 of SO3 embedding $h_a, [N, so3]$
  - Incorporate initial edge information (distance RBF + bond features + initial atom embeddings):
    - $h_a +=f(R', f_e', f_a'), [N,so3]$
    - This is done by projecting edge attrs, src and dst attr, distance with eg. an MLP to represent the message, rotate the embeddings to the edge direction, assign this new hidden state to the m=0 component of the SO3 embedding, and then rotate the whole embedding to the edge direction.
  - Mix the l=1 features at this point with the 3 features from topology:
    - $h_a[:, 1:4, :] = mix(h_a[:, 1:4, :], f_a^{l1})$

5. We now have the starting state of the SO3 node embeddings that will be updated over multiple attention layers.
  - Layer norm equivalent incoming embeddings
  - Here $h_a = h_a + \Sigma_j^{Neighbors}\alpha_j m_j$
  - The message is first constructed as a function of the two node hidden states and edge/node attributes:
    - Feature importance computed as a function of edge and node ATTRIBUTES (eg. layer dependent, not the original embeddings): $w_m=Lin(f_a^j,f_a^i,f_e^{ij},d^{ij})$ each w maps to a particular spherical harmonic component (irrespective of spin state)
    - Raw message computed as a function of the two so3 embeddings $m_j' = Lin(h_a^j, h_a^i)$
    - Weigh  each spherical harmonic component in the message $m_j' = m_j'* w_m$ These are difference shapes - there is some complex broadcasting such that eg. all m components of a singl l1 feature of a single channel of the message has its own weight.
    - Convolve the message across different angular momenta for m-orders that align: $m_j' = Convolve(m_j')$ Includes linear mixing across l
    - At this point each message is only a function of the src, node, and edge - not the full context of the target node.
    - Use the "extra" l0 features of the message hidden state to compute attention logit for the src node: $\alpha_j = Softmax(Lin(m_j'^extra[:, 0, :]))$
  - Apply learned convolution (as opposed to having coefficient weights being determnined by edges)
    - $m_j = Convolve_{learnable}(m_j')$
  - Update the node hidden state:
    - $h_a = h_a + \Sigma_j^{Neighbors}\alpha_j m_j$
  - Non linear activation after each convolution
  - MLPSO3 layer to update the hidden state:
    - $h_a = MLP_So3(h_a)$
  - Repeat the above for a number of layers



In [106]:
from typing import List, Dict, Any, Optional

import torch
from torch import nn
import numpy as np

import copy

In [107]:
NUM_CHANNELS = 64
LMAX_LIST = [3]
MMAX_LIST = [2]
NUM_HEADS = 4

NUM_RESOLUTIONS = len(LMAX_LIST)
SPHERE_CHANNELS_ALL = NUM_RESOLUTIONS * NUM_CHANNELS

### 0. Load the data

In [108]:
from metalsitenn.dataloading import MetalSiteDataset
from metalsitenn.featurizer import MetalSiteCollator
from torch.utils.data import DataLoader
from metalsitenn.utils import visualize_protein_data_3d

In [109]:
ds = MetalSiteDataset(
    cache_folder='../../metal_site_modeling/data/1/1.1_parse_sites_metadata',
)

In [136]:
collator = MetalSiteCollator(
    atom_features=['element', 'charge', 'nhyd', 'hyb'],
    bond_features=['bond_order', 'is_in_ring', 'is_aromatic'],
    metal_unknown=False,
    metal_classification=True,
    residue_collapse_do=True,
    residue_collapse_time=0.2,
)

In [137]:
loader = DataLoader(
    ds,
    batch_size=4,
    collate_fn=collator,
    shuffle=True,
    num_workers=4,
)

In [138]:
batch = next(iter(loader))

In [139]:
assert batch.distance_vec is not None

In [140]:
feature_vocab_sizes = collator.featurizer.get_feature_vocab_sizes()
feature_vocab_sizes

{'element': 46,
 'charge': 8,
 'nhyd': 7,
 'hyb': 8,
 'bond_order': 6,
 'is_in_ring': 3,
 'is_aromatic': 3}

### 0.5 - Init some equivariant related classes necessary throughout the model

In [141]:
from fairchem.core.models.equiformer_v2.so3 import (
    CoefficientMappingModule,
    SO3_Embedding,
    SO3_Grid,
    SO3_LinearV2,
    SO3_Rotation,
)
from fairchem.core.models.equiformer_v2.layer_norm import get_normalization_layer


In [142]:
mappingReduced = CoefficientMappingModule(
    lmax_list=LMAX_LIST,
    mmax_list=MMAX_LIST)

In [143]:
SO3_rotation = nn.ModuleList()
for i in range(NUM_RESOLUTIONS):
    SO3_rotation.append(SO3_Rotation(LMAX_LIST[i]))

In [144]:
src_pos = batch.positions[batch.edge_index[:,0]]
dst_pos = batch.positions[batch.edge_index[:,1]]
edge_distance_vector = dst_pos - src_pos

In [145]:
from fairchem.core.models.equiformer_v2.edge_rot_mat import init_edge_rot_mat
edge_rot_mat = init_edge_rot_mat(edge_distance_vector)

In [146]:
for sor_rot in SO3_rotation:
    sor_rot.set_wigner(edge_rot_mat)

In [147]:
from fairchem.core.models.equiformer_v2.module_list import ModuleListInfo

SO3_grid = ModuleListInfo(
    f"({max(LMAX_LIST)}, {max(LMAX_LIST)})"
)
for lval in range(max(LMAX_LIST) + 1):
    SO3_m_grid = nn.ModuleList()
    for m in range(max(LMAX_LIST) + 1):
        SO3_m_grid.append(
            SO3_Grid(
                lval,
                m,
                resolution=None,
                normalization="component",
            )
        )
    SO3_grid.append(SO3_m_grid)

### 1. Node and edge embeddings into attributes

In [148]:
class NodeEmbedder(nn.Module):
    """
    Embed atom features, concat, then project to output dimension.
    
    Handles all atom-level features in the molecular graph.
    
    Args:
        feature_vocab_sizes: Dict mapping atom feature names to vocab sizes
        atom_features: List of atom feature names
        output_dim: Output dimension for concatenated atom features
        embedding_dim: Individual embedding dimension per feature
        use_bias: Whether to use bias in final projection layer
    """
    
    def __init__(
        self,
        feature_vocab_sizes: Dict[str, int],
        atom_features: List[str],
        output_dim: int = 64,
        embedding_dim: int = 32,
        use_bias: bool = True
    ):
        super().__init__()
        
        self.atom_features = atom_features
        
        # Separate embeddings for each atom feature
        self.embeddings = nn.ModuleDict()
        for feature in atom_features:
            vocab_size = feature_vocab_sizes[feature]
            self.embeddings[feature] = nn.Embedding(vocab_size, embedding_dim)
            
        # Project concatenated features to desired output dimension
        concat_dim = len(atom_features) * embedding_dim
        self.projection = nn.Linear(concat_dim, output_dim, bias=use_bias)
        
    def forward(self, feature_dict: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Embed and project atom features.
        
        Args:
            feature_dict: Dict of feature_name -> token_indices tensor
            
        Returns:
            Atom embeddings tensor of shape (num_atoms, output_dim)
        """
        # Embed each atom feature individually
        embedded_features = []
        for name in self.atom_features:
            tokens = feature_dict[name]
            embedded = self.embeddings[name](tokens.squeeze(-1))
            embedded_features.append(embedded)
            
        # Concatenate all atom features
        atom_embeds = torch.cat(embedded_features, dim=-1)
        
        # Project to output dimension
        atom_embeds = self.projection(atom_embeds)
        return atom_embeds


class EdgeEmbedder(nn.Module):
    """
    Embed bond features, concat, then project to output dimension.
    
    Handles all edge-level features in the molecular graph.
    
    Args:
        feature_vocab_sizes: Dict mapping bond feature names to vocab sizes
        bond_features: List of bond feature names
        output_dim: Output dimension for concatenated bond features
        embedding_dim: Individual embedding dimension per feature
        use_bias: Whether to use bias in final projection layer
    """
    
    def __init__(
        self,
        feature_vocab_sizes: Dict[str, int],
        bond_features: List[str],
        output_dim: int = 64,
        embedding_dim: int = 32,
        use_bias: bool = True
    ):
        super().__init__()
        
        self.bond_features = bond_features
        
        # Separate embeddings for each bond feature
        self.embeddings = nn.ModuleDict()
        for feature in bond_features:
            vocab_size = feature_vocab_sizes[feature]
            self.embeddings[feature] = nn.Embedding(vocab_size, embedding_dim)
            
        # Project concatenated features to desired output dimension
        concat_dim = len(bond_features) * embedding_dim
        self.projection = nn.Linear(concat_dim, output_dim, bias=use_bias)
        
    def forward(self, feature_dict: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Embed and project bond features.
        
        Args:
            feature_dict: Dict of feature_name -> token_indices tensor
            
        Returns:
            Bond embeddings tensor of shape (num_bonds, output_dim)
        """
        # Embed each bond feature individually
        embedded_features = []
        for name in self.bond_features:
            tokens = feature_dict[name]
            embedded = self.embeddings[name](tokens.squeeze(-1))
            embedded_features.append(embedded)
            
        # Concatenate all bond features
        bond_embeds = torch.cat(embedded_features, dim=-1)
        
        # Project to output dimension
        bond_embeds = self.projection(bond_embeds)
        return bond_embeds

In [149]:
l0nodeembedder = NodeEmbedder(
    feature_vocab_sizes=feature_vocab_sizes,
    atom_features=['element', 'charge', 'nhyd', 'hyb'],
    output_dim=SPHERE_CHANNELS_ALL,
    embedding_dim=32)

In [150]:
feature_dict = {}
for feature in collator.featurizer.atom_features:
    if getattr(batch, feature) is not None:
        feature_dict[feature] = getattr(batch, feature)
    else:
        raise ValueError(f"Feature {feature} not found in batch")

for feature in collator.featurizer.bond_features:
    if getattr(batch, feature) is not None:
        feature_dict[feature] = getattr(batch, feature)
    else:
        raise ValueError(f"Feature {feature} not found in batch")

In [151]:
node_attributes = l0nodeembedder(feature_dict)

In [152]:
print("Node attributes shape:", node_attributes.shape)

Node attributes shape: torch.Size([367, 64])


### 2. Radial basis expansion of edge distances

In [153]:
from fairchem.core.models.scn.smearing import GaussianSmearing
start = 0.0
stop = 7.0
num_basis = 24

radial_basis = GaussianSmearing(
    start=0.0,
    stop=7.0,
    num_gaussians=24,
    basis_width_scalar=(stop - start) / num_basis
)

In [154]:
R = radial_basis(batch.distances)

In [155]:
print("Distance RBF shape:", R.shape)

Distance RBF shape: torch.Size([7340, 24])


### 3. Compute the l1 feature vectors based on topology 

In [156]:
from metalsitenn.placer_modules.losses import bondLoss
from metalsitenn.placer_modules.geometry import triple_prod

def compute_positional_topology_gradients(
    r: torch.Tensor,
    bond_indexes: torch.Tensor,
    bond_lengths: torch.Tensor,
    chirals: torch.Tensor,
    planars: torch.Tensor,
    gclip: float = 100.0,
    atom_mask: Optional[torch.Tensor] = None,
):
    """Get gradients of positions with respect to topology features.

    A la. ChemNet ; https://github.com/baker-laboratory/PLACER/blob/main/modules/model.py

    Some additions:
    - the gradient is flipped in direction such that it makes physical sense - these vectors point in the direction the atom should
      move. This should make no difference for downstream neural operations as weights can flip anyway.
    - option to provide mask for atoms, which will zero out gradients for masked atoms. This is useful for training with masked atoms.

    Args:
        r (torch.Tensor): Atom positions, shape (N, 3).
        bonds (torch.Tensor): Bond indexes, shape (M, 2).
        bond_lengths (torch.Tensor): Bond lengths, shape (M,1).
        chirals (torch.Tensor): Chirality features, shape (O,5).
        planars (torch.Tensor): Planarity features, shape (P,5).
        gclip (float): Gradient clipping value.
        atom_mask (torch.Tensor, optional): Mask for atoms, shape (N,). If provided, gradients will be zeroed for masked atoms.

    Returns:
        grads (torch.Tensor): Gradients of shape (N, 3, 3). (vectors from each of both length, chirals, planars).
    """
    N = r.shape[0]
    device = r.device

    with torch.enable_grad():
        r_detached = r.detach() # so that the computation graph does not include the result of this function, which is essentially external context / input
        r_detached.requires_grad = True  # Enable gradients for positions

        g = torch.zeros((N, 3, 3), device=device)
    
        # Compute bond gradients
        if len(bond_indexes) > 0:
            l = bondLoss(
                r_detached,
                ij=bond_indexes,
                b0=bond_lengths,
                mean=False
            )
            g[:, 0] = torch.autograd.grad(l, r_detached)[0].data

        # Compute chirality gradients
        if len(chirals) > 0:
            o,i,j,k = r_detached[chirals].permute(1, 0, 2)
            l = ((triple_prod(o-i,o-j,o-k,norm=True)-0.70710678)**2).sum()
            g[:, 1] = torch.autograd.grad(l, r_detached)[0].data

        # Compute planarity gradients
        if len(planars) > 0:
            o,i,j,k = r_detached[planars].permute(1, 0, 2)
            l = ((triple_prod(o-i,o-j,o-k,norm=True)**2).sum())
            g[:, 2] = torch.autograd.grad(l, r_detached)[0].data

        # Scale and clip
        g = torch.nan_to_num(g, nan=0.0, posinf=gclip, neginf=-gclip)
        gnorm = torch.linalg.norm(g, dim=-1)
        mask = gnorm > gclip
        g[mask] /= gnorm[mask][...,None]
        g[mask]  *= gclip

        # flip direction of gradients
        g = -g

        # Zero gradients for masked atoms
        if atom_mask is not None:
            g *= atom_mask[:, None, None].to(g.dtype)

        # reorder such that the three types of gradients (channels) are the last dim, instead of the vector being there
        g = g.permute(0, 2, 1)

        return g.detach()

In [157]:
# make sure to mask out any masked atoms
masked_elements = batch.element == collator.featurizer.tokenizers['element'].mask_token_id
if sum(masked_elements) == 0:
    atom_mask = None
else:
    atom_mask = ~masked_elements

node_l1_gradients = compute_positional_topology_gradients(
    r=batch.positions,
    bond_indexes=batch.topology['bonds'],
    bond_lengths=batch.topology['bond_lengths'],
    chirals=batch.topology['chirals'],
    planars=batch.topology['planars'],
    gclip=1000.0,
    atom_mask=atom_mask
)

In [158]:
print("Node l1 gradients shape:", node_l1_gradients.shape)  

Node l1 gradients shape: torch.Size([367, 3, 3])


In [159]:
# show some gradients on the data
prot = batch[0]
n_atoms = len(prot.positions)
print("Number of atoms in first protein:", n_atoms)

Number of atoms in first protein: 77


In [160]:
l1_1st_p = node_l1_gradients[:n_atoms,:,:]

In [161]:
visualize_protein_data_3d(prot, velocities=l1_1st_p[:,:,0])

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Note that the third dimension here is channels

### 4. Generate SO3 initial

#### First just send the node attributes onto the l=0 component of the SO3 embedding

In [78]:
from fairchem.core.models.equiformer_v2.so3 import SO3_Embedding, SO3_LinearV2

In [79]:
class SO3ScalarEmbedder(nn.Module):
    """
    Converts pre-computed atom embeddings to SO3 embeddings by projecting them 
    to the l=0, m=0 coefficients across multiple resolutions.
    
    Args:
        input_dim (int): Dimension of input atom embeddings
        lmax_list (list[int]): List of maximum degrees (l) for each resolution
        sphere_channels (int): Number of spherical channels per resolution
        device (str): Device to place tensors on
    """
    
    def __init__(
        self,
        lmax_list: list[int],
        sphere_channels: int,
    ):
        super().__init__()
        
        self.lmax_list = lmax_list
        self.sphere_channels = sphere_channels
        self.num_resolutions = len(lmax_list)
        self.sphere_channels_all = self.num_resolutions * sphere_channels

    def forward(self, atom_embeddings: torch.Tensor) -> SO3_Embedding:
        """
        Convert atom embeddings to SO3 embeddings.
        
        Args:
            atom_embeddings (torch.Tensor): Input atom embeddings of shape (N, sphere_channels_all)
            
        Returns:
            SO3_Embedding: SO3 embedding with l=0, m=0 coefficients initialized
        """
        num_atoms = atom_embeddings.shape[0]
        if atom_embeddings.shape[1] != self.sphere_channels_all:
            raise ValueError(
                f"Expected atom_embeddings shape (N, {self.sphere_channels_all}), "
                f"but got {atom_embeddings.shape}"
            )
        
        # Initialize SO3 embedding
        x = SO3_Embedding(
            num_atoms,
            self.lmax_list,
            self.sphere_channels,
            atom_embeddings.device,
            atom_embeddings.dtype,
        )
        
        # Fill in the l=0, m=0 coefficients for each resolution
        offset_res = 0  # Offset in SO3 embedding coefficient dimension
        offset_channels = 0  # Offset in projected embedding channels
        
        for i in range(self.num_resolutions):
            if self.num_resolutions == 1:
                # Single resolution case - use all projected channels
                x.embedding[:, offset_res, :] = atom_embeddings
            else:
                # Multi-resolution case - split channels across resolutions
                x.embedding[:, offset_res, :] = atom_embeddings[
                    :, offset_channels : offset_channels + self.sphere_channels
                ]
            
            # Update offsets for next resolution
            offset_channels += self.sphere_channels
            offset_res += int((self.lmax_list[i] + 1) ** 2)
        
        return x


In [80]:
node_scaler_embedder = SO3ScalarEmbedder(
    lmax_list=LMAX_LIST,
    sphere_channels=NUM_CHANNELS,
)

In [81]:
h_a = node_scaler_embedder(node_attributes)

In [82]:
(h_a.embedding[:,0] == node_attributes).all()

tensor(True)

#### Generate edge embeddings and assign them to the m=0 component of the SO3 embedding

##### First a general method for taking distance, node AND edge attributes and projecting them to target size

In [83]:
from fairchem.core.models.equiformer_v2.radial_function import RadialFunction 

class EdgeProjector(nn.Module):
    """
    Embed edges to output of target size.

    In equiformer, the radial basis distance is optionally combined with a src and dst node embedding,
    then projected with an MLP "radial_func" to a target size - the target size depends on the application.
    This class is meant to extract that functionality out and allow us to use all the extra node and edge features we
    have beyond just the distance and atomic identity.

    Args:
        radial_basis_size (int): Size of RBF expected
        feature_vocab_sizes (Dict[str, int]): Dictionary mapping feature names to vocab sizes
        use_edge_features (bool): Whether to use edge features
        bond_features (List[str]): List of bond feature names to use
        use_node_features (bool): Whether to use node features
        node_features (List[str]): List of node feature names to use
        output_dim (int): Output dimension for the edge embeddings
        embedding_dim (int): Embedding dimension for node and edge features from NodeEmbedder and EdgeEmbedder
        embedding_use_bias (bool): Whether to use bias in the embedding layers
        projector_hidden_layers (int): Number of hidden layers in the projector Radial func
        projector_output_size (int): Output size of the projector Radial func

    """
    
    def __init__(
        self,
        radial_basis_size: int,
        feature_vocab_sizes: Dict[str, int]={},
        use_edge_features: bool=True,
        bond_features: List[str]=['bond_order', 'is_in_ring', 'is_aromatic'],
        use_node_features: bool=True,
        node_features: List[str]=['element', 'charge', 'nhyd', 'hyb'],
        output_dim: int = 64,
        embedding_dim: int = 32,
        embedding_use_bias: bool = True,
        use_projector: bool = True,
        projector_hidden_layers: int = 1,
        projector_size: int = 64
    ):
        super().__init__()

        self.radial_basis_size = radial_basis_size
        self.feature_vocab_sizes = feature_vocab_sizes
        self.use_edge_features = use_edge_features
        self.bond_features = bond_features
        self.use_node_features = use_node_features
        self.node_features = node_features
        self.output_dim = output_dim
        self.embedding_dim = embedding_dim
        self.use_bias = embedding_use_bias
        self.projector_hidden_layers = projector_hidden_layers
        self.projector_size = projector_size
        self.use_projector = use_projector

        # if we are using node features, create embedders
        if self.use_node_features:
            self.source_embedding = NodeEmbedder(
                feature_vocab_sizes=self.feature_vocab_sizes,
                atom_features=self.node_features,
                output_dim=embedding_dim,
                embedding_dim=embedding_dim,
                use_bias=embedding_use_bias
            )
            self.destination_embedding = NodeEmbedder(
                feature_vocab_sizes=self.feature_vocab_sizes,
                atom_features=self.node_features,
                output_dim=embedding_dim,
                embedding_dim=embedding_dim,
                use_bias=embedding_use_bias
            )
        else:
            self.source_embedding = None
            self.destination_embedding = None

        # if we are using edge features, create embedder
        if self.use_edge_features:
            self.edge_embedding = EdgeEmbedder(
                feature_vocab_sizes=self.feature_vocab_sizes,
                bond_features=self.bond_features,
                output_dim=embedding_dim,
                embedding_dim=embedding_dim,
                use_bias=embedding_use_bias
            )
        else:
            self.edge_embedding = None

        # get the epected input size for the radial function
        input_size = radial_basis_size
        if self.use_edge_features:
            input_size += embedding_dim
        if self.use_node_features:
            input_size += 2 * embedding_dim

        self.input_size = input_size

        # radial function to project the input to the output dimension
        if self.use_projector:
            channels_list = [input_size] + [self.projector_size] * self.projector_hidden_layers + [self.output_dim]
            self.radial_func = RadialFunction(
                channels_list=channels_list,
            )

    def forward(
        self,
        R: torch.Tensor, # [E, radial_basis_size]
        edge_index: torch.Tensor, # [E,2]
        feature_dict: Dict[str, torch.Tensor]={},
    ):
        to_concat = []
        # radial basis distance
        to_concat.append(R)

        # edge features
        if self.use_edge_features:
            edge_features = self.edge_embedding(feature_dict)
            to_concat.append(edge_features)

        # node features
        if self.use_node_features:
            nodes_embedded = self.source_embedding(feature_dict)

            # Extract the source and destination node embeddings
            src_embeddings = nodes_embedded[edge_index[:, 0]]
            dst_embeddings = nodes_embedded[edge_index[:, 1]]
            # Concatenate source and destination node embeddings
            to_concat.append(src_embeddings)
            to_concat.append(dst_embeddings)

        # concatenate all features
        concatenated = torch.cat(to_concat, dim=-1)
        if self.use_projector:
            # pass through radial function
            output = self.radial_func(concatenated)

            return output
        else:
            return concatenated

In [84]:
# just a quick test of the class, we do not know the required output size yet
edge_projector = EdgeProjector(
    radial_basis_size=R.shape[1],
    feature_vocab_sizes=feature_vocab_sizes,
    use_edge_features=True,
    bond_features=collator.featurizer.bond_features,
    use_node_features=True,
    node_features=collator.featurizer.atom_features,
    output_dim=5,
    embedding_dim=32,
    embedding_use_bias=True,
    projector_hidden_layers=1,
    projector_size=64
)
unusable_edge_features = edge_projector(
    R,
    edge_index=batch.edge_index,
    feature_dict=feature_dict
)

##### Now we use it inside a modified EdgeDegreeEmbedding class from eqf2
https://github.com/facebookresearch/fairchem/blob/977a80328f2be44649b414a9907a1d6ef2f81e95/src/fairchem/core/models/equiformer_v2/input_block.py#L12

Difference is here we have additional information for our invariant edge features.

In [85]:
class EdgeDegreeEmbedding(torch.nn.Module):
    """

    Args:
        sphere_channels (int):      Number of spherical channels

        lmax_list (list:int):       List of degrees (l) for each resolution
        mmax_list (list:int):       List of orders (m) for each resolution

        SO3_rotation (list:SO3_Rotation): Class to calculate Wigner-D matrices and rotate embeddings
        mappingReduced (CoefficientMappingModule): Class to convert l and m indices once node embedding is rotated

        DEPRECATED, using EdgeProjector instead
        # max_num_elements (int):     Maximum number of atomic numbers
        # edge_channels_list (list:int):  List of sizes of invariant edge embedding. For example, [input_channels, hidden_channels, hidden_channels].
                                        The last one will be used as hidden size when `use_atom_edge_embedding` is `True`.
        # use_atom_edge_embedding (bool): Whether to use atomic embedding along with relative distance for edge scalar features
        radial_basis_size (int):     Number of radial basis functions expected
        feature_vocab_sizes (list:int): List of sizes of feature vocabularies
        use_edge_features (bool):    Whether to use edge features
        bond_features (list:str): List of bond feature names to use if using any
        use_node_features (bool): Whether to use node features
        node_features (list:str): List of node feature names to use if using any
        embedding_dim (int):        Embedding dimension for node and edge features
        embedding_use_bias (bool):  Whether to use bias in the embedding layers
        projector_hidden_layers (int): Number of hidden layers in the projector Radial func
        projector_size (int):       Hidden layer size of the projector Radial func
        NOTE: Output size of radial func is determined by number of m0 coefficients available.

        rescale_factor (float):     Rescale the sum aggregation
    """

    def __init__(
        self,
        sphere_channels: int,
        lmax_list: list[int],
        mmax_list: list[int],
        SO3_rotation,
        mappingReduced,
        radial_basis_size: int,
        feature_vocab_sizes: Dict[str, int]={},
        use_edge_features: bool=True,
        bond_features: List[str]=['bond_order', 'is_in_ring', 'is_aromatic'],
        use_node_features: bool=True,
        node_features: List[str]=['atomic_number', 'formal_charge'],
        embedding_dim: int=128,
        embedding_use_bias: bool=True,
        projector_hidden_layers: int=2,
        projector_size: int=64,
        rescale_factor: float=1.0,
    ):
        super().__init__()
        self.sphere_channels = sphere_channels
        self.lmax_list = lmax_list
        self.mmax_list = mmax_list
        self.num_resolutions = len(self.lmax_list)
        self.SO3_rotation = SO3_rotation
        self.mappingReduced = mappingReduced

        self.m_0_num_coefficients: int = self.mappingReduced.m_size[0]
        self.m_all_num_coefficents: int = len(self.mappingReduced.l_harmonic)

        # output size as
        rad_output_size = self.m_0_num_coefficients * self.sphere_channels
        self.rad_func = EdgeProjector(
            radial_basis_size=radial_basis_size,
            feature_vocab_sizes=feature_vocab_sizes,
            use_edge_features=use_edge_features,
            bond_features=bond_features,
            use_node_features=use_node_features,
            node_features=node_features,
            output_dim=rad_output_size,
            embedding_dim=embedding_dim,
            embedding_use_bias=embedding_use_bias,
            projector_hidden_layers=projector_hidden_layers,
            projector_size=projector_size
        )

        self.rescale_factor = rescale_factor

    def forward(
        self, 
        edge_distance_rbf: torch.Tensor,
        edge_index: torch.Tensor, 
        num_nodes: int, 
        feature_dict: Dict[str, torch.Tensor] = {},
        node_offset: int = 0
    ):
        """
        Forward pass for edge degree embedding.
        
        Args:
            edge_distance_rbf (torch.Tensor): Radial basis function expansion of edge distances [E, radial_basis_size]
            edge_index (torch.Tensor): Edge indices [2, E]
            num_nodes (int): Number of nodes in the graph
            feature_dict (Dict[str, torch.Tensor]): Dictionary containing node and edge features
            node_offset (int): Offset for node indices (default: 0)
            
        Returns:
            SO3_Embedding: Edge embedding in SO3 format
        """
        # Use EdgeProjector to compute edge features including distance, node features, and edge features
        x_edge_m_0 = self.rad_func(edge_distance_rbf, edge_index, feature_dict)
        
        # Reshape to [num_edges, m_0_coefficients, sphere_channels]
        x_edge_m_0 = x_edge_m_0.reshape(
            -1, self.m_0_num_coefficients, self.sphere_channels
        )
        
        # Pad with zeros for higher m coefficients
        x_edge_m_pad = torch.zeros(
            (
                x_edge_m_0.shape[0],
                (self.m_all_num_coefficents - self.m_0_num_coefficients),
                self.sphere_channels,
            ),
            device=x_edge_m_0.device,
        )
        x_edge_m_all = torch.cat((x_edge_m_0, x_edge_m_pad), dim=1)

        # Create SO3 embedding
        x_edge_embedding = SO3_Embedding(
            0,
            self.lmax_list.copy(),
            self.sphere_channels,
            device=x_edge_m_all.device,
            dtype=x_edge_m_all.dtype,
        )
        x_edge_embedding.set_embedding(x_edge_m_all)
        x_edge_embedding.set_lmax_mmax(self.lmax_list.copy(), self.mmax_list.copy())

        # Reshape the spherical harmonics based on l (degree)
        x_edge_embedding._l_primary(self.mappingReduced)

        # Rotate back the irreps
        x_edge_embedding._rotate_inv(self.SO3_rotation, self.mappingReduced)

        # Compute the sum of the incoming neighboring messages for each target node
        x_edge_embedding._reduce_edge(edge_index[:,1] - node_offset, num_nodes)
        x_edge_embedding.embedding = x_edge_embedding.embedding / self.rescale_factor

        return x_edge_embedding

In [86]:
edge_deg_embedder = EdgeDegreeEmbedding(
    sphere_channels=NUM_CHANNELS,
    lmax_list=LMAX_LIST,
    mmax_list=MMAX_LIST,
    SO3_rotation=SO3_rotation,
    mappingReduced=mappingReduced,
    radial_basis_size=R.shape[1],
    feature_vocab_sizes=feature_vocab_sizes,
    use_edge_features=True,
    bond_features=collator.featurizer.bond_features,
    use_node_features=True,
    node_features=collator.featurizer.atom_features,
    embedding_dim=32,
    embedding_use_bias=True,
    projector_hidden_layers=1,
    projector_size=64,
    rescale_factor=1.0
)

In [87]:
h_a_edges = edge_deg_embedder(
    edge_distance_rbf=R,
    edge_index=batch.edge_index,
    num_nodes=batch.positions.shape[0],
    feature_dict=feature_dict,
    node_offset=0
)

In [88]:
h_a.embedding = h_a.embedding + h_a_edges.embedding

In [89]:
input_norm_1 = get_normalization_layer(
    'layer_norm_sh',
    lmax = max(LMAX_LIST),
    num_channels=SPHERE_CHANNELS_ALL
)


In [90]:
h_a.embedding = input_norm_1(h_a.embedding)

#### Mixing to create final SO3 embeddings before attention layers

In [91]:
import torch
import torch.nn as nn
import math
from typing import List, Optional

class SO3_L1_Linear(nn.Module):
    """
    Equivariant linear layer that operates only on L=1 spherical harmonic features.
    Maintains SO(3) equivariance by using shared weights for all m components.
    
    Input: [N, 3, in_channels] -> Output: [N, 3, out_channels]
    
    Args:
        in_channels: Input feature channels
        out_channels: Output feature channels  
        bias: Whether to use bias term (should be False for L=1 to maintain equivariance)
    """
    
    def __init__(self, in_channels: int, out_channels: int, bias: bool = False):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        # Single weight matrix shared by all m components of L=1
        self.weight = nn.Parameter(torch.randn(out_channels, in_channels))
        bound = 1 / math.sqrt(in_channels)
        nn.init.uniform_(self.weight, -bound, bound)
        
        if bias:
            raise ValueError("Bias should be False for L=1 to maintain equivariance")
        
    def forward(self, x):
        """
        Args:
            x: [N, 3, in_channels] L=1 features (m=-1,0,1 components)
        Returns:
            [N, 3, out_channels] transformed L=1 features
        """
        # Apply same linear transformation to all m components
        # x @ weight.T maintains equivariance since all m share same weights
        return torch.einsum('nmi, oi -> nmo', x, self.weight)
    

class SO3_L1_LinearMixing(nn.Module):
    """
    Applies a linear transformation to L=1 spherical harmonic features.
    
    This layer is designed to mix L=1 features while maintaining equivariance.
    
    Args:
        in_channels_list: List of input sizes for each item to be mixed
        out_channels: Output size for the mixed features
    """
    def __init__(self, in_channels_list: List[int], out_channels: int):
        super().__init__()
        
        self.in_channels_list = in_channels_list
        self.out_channels = out_channels
        
        # Create a linear layer for each input channel size
        self.linears = nn.ModuleList([
            SO3_L1_Linear(in_channels, out_channels) for in_channels in in_channels_list
        ])
        
    def forward(self, x_list: List[torch.Tensor]) -> torch.Tensor:
        """
        Args:
            x_list: List of tensors with shape [N, 3, in_channels_i] for each input
        Returns:
            Tensor with shape [N, 3, out_channels] after mixing
        """
        mixed_features = [linear(x) for linear, x in zip(self.linears, x_list)]
        return torch.stack(mixed_features, dim=2).sum(dim=2)

In [92]:
mixer = SO3_L1_LinearMixing(
    in_channels_list=[SPHERE_CHANNELS_ALL, 3],
    out_channels=SPHERE_CHANNELS_ALL
)
current_l1_features = h_a.embedding[:, 1:4, :]  # Extract L=1 features (m=-1,0,1)

mixed_features = mixer([current_l1_features, node_l1_gradients])

In [93]:
h_a.embedding[:, 1:4, :] = mixed_features

### 5. Update the SO3 embeddings with attention layers

First the SO2 attention has to be updated to accept edge information

In [94]:
import torch_geometric

from fairchem.core.common import gp_utils

from fairchem.core.models.equiformer_v2.activation import (
    GateActivation,
    S2Activation,
    SeparableS2Activation,
    SmoothLeakyReLU,
)
from fairchem.core.models.equiformer_v2.drop import EquivariantDropoutArraySphericalHarmonics, GraphDropPath
from fairchem.core.models.equiformer_v2.layer_norm import get_normalization_layer
from fairchem.core.models.equiformer_v2.radial_function import RadialFunction
from fairchem.core.models.equiformer_v2.so2_ops import SO2_Convolution
from fairchem.core.models.equiformer_v2.so3 import SO3_Embedding, SO3_LinearV2


class SO2EquivariantGraphAttentionWEdgesV2(torch.nn.Module):
    """
    SO2EquivariantGraphAttention: Perform MLP attention + non-linear message passing
        SO(2) Convolution with radial function -> S2 Activation -> SO(2) Convolution -> attention weights and non-linear messages
        attention weights * non-linear messages -> Linear

    Args:
        sphere_channels (int):      Number of spherical channels
        hidden_channels (int):      Number of hidden channels used during the SO(2) conv
        num_heads (int):            Number of attention heads
        attn_alpha_channels (int):  Number of channels for alpha vector in each attention head
        attn_value_channels (int):  Number of channels for value vector in each attention head
        output_channels (int):      Number of output channels
        lmax_list (list:int):       List of degrees (l) for each resolution
        mmax_list (list:int):       List of orders (m) for each resolution

        SO3_rotation (list:SO3_Rotation): Class to calculate Wigner-D matrices and rotate embeddings
        mappingReduced (CoefficientMappingModule): Class to convert l and m indices once node embedding is rotated
        SO3_grid (SO3_grid):        Class used to convert from grid the spherical harmonic representations

        edge_channels_list (list:int): List of sizes of invariant edge embedding. For example, [input_channels, hidden_channels, hidden_channels].
        use_m_share_rad (bool):     Whether all m components within a type-L vector of one channel share radial function weights

        # for EdgeProjector - replaces use_atom_edge_embedding and related parameters
        use_edge_information (bool): Whether to use edge information in the attention mechanism
        radial_basis_size (int):     Number of radial basis functions expected
        feature_vocab_sizes (Dict[str, int]): Dictionary mapping feature names to vocab sizes
        use_edge_features (bool):    Whether to use edge features
        bond_features (List[str]):   List of bond feature names to use if using any
        use_node_features (bool):    Whether to use node features
        node_features (List[str]):   List of node feature names to use if using any
        embedding_dim (int):         Embedding dimension for node and edge features
        embedding_use_bias (bool):   Whether to use bias in the embedding layers

        activation (str):           Type of activation function
        use_s2_act_attn (bool):     Whether to use attention after S2 activation. Otherwise, use the same attention as Equiformer
        use_attn_renorm (bool):     Whether to re-normalize attention weights
        use_gate_act (bool):        If `True`, use gate activation. Otherwise, use S2 activation.
        use_sep_s2_act (bool):      If `True`, use separable S2 activation when `use_gate_act` is False.

        alpha_drop (float):         Dropout rate for attention weights
    """

    def __init__(
        self,
        sphere_channels: int,
        hidden_channels: int,
        num_heads: int,
        attn_alpha_channels: int,
        attn_value_channels: int,
        output_channels: int,
        lmax_list: list[int],
        mmax_list: list[int],
        SO3_rotation,
        mappingReduced,
        SO3_grid,
        edge_channels_list,
        use_m_share_rad: bool = False,
        # EdgeProjector parameters
        use_edge_information: bool = True,
        radial_basis_size: int = 50,
        feature_vocab_sizes: Dict[str, int] = None,
        use_edge_features: bool = True,
        bond_features: List[str] = None,
        use_node_features: bool = True,
        node_features: List[str] = None,
        embedding_dim: int = 32,
        embedding_use_bias: bool = True,
        activation="scaled_silu",
        use_s2_act_attn: bool = False,
        use_attn_renorm: bool = True,
        use_gate_act: bool = False,
        use_sep_s2_act: bool = True,
        alpha_drop: float = 0.0,
    ):
        super().__init__()

        self.sphere_channels = sphere_channels
        self.hidden_channels = hidden_channels
        self.num_heads = num_heads
        self.attn_alpha_channels = attn_alpha_channels
        self.attn_value_channels = attn_value_channels
        self.output_channels = output_channels
        self.lmax_list = lmax_list
        self.mmax_list = mmax_list
        self.num_resolutions = len(self.lmax_list)

        self.SO3_rotation = SO3_rotation
        self.mappingReduced = mappingReduced
        self.SO3_grid = SO3_grid

        # Edge feature processing
        self.use_edge_information = use_edge_information
        self.use_m_share_rad = use_m_share_rad
        
        if feature_vocab_sizes is None:
            feature_vocab_sizes = {}
        if bond_features is None:
            bond_features = ['bond_order', 'is_in_ring', 'is_aromatic']
        if node_features is None:
            node_features = ['element', 'charge', 'nhyd', 'hyb']

        # Initialize edge projector
        if self.use_edge_information:
            self.edge_projector = EdgeProjector(
                radial_basis_size=radial_basis_size,
                feature_vocab_sizes=feature_vocab_sizes,
                use_edge_features=use_edge_features,
                bond_features=bond_features,
                use_node_features=use_node_features,
                node_features=node_features,
                output_dim=edge_channels_list[-1],  # Match expected output dimension
                embedding_dim=embedding_dim,
                embedding_use_bias=embedding_use_bias,
                use_projector=False,  # Just concatenation, no radial function
                projector_hidden_layers=1,
                projector_size=64
            )
            # Update edge channels list input size based on projector concatenated size
            self.edge_channels_list = copy.deepcopy(edge_channels_list)
            self.edge_channels_list[0] = self.edge_projector.input_size
        else:
            self.edge_projector = None
            self.edge_channels_list = copy.deepcopy(edge_channels_list)

        self.use_s2_act_attn = use_s2_act_attn
        self.use_attn_renorm = use_attn_renorm
        self.use_gate_act = use_gate_act
        self.use_sep_s2_act = use_sep_s2_act

        assert not self.use_s2_act_attn  # since this is not used

        # Create SO(2) convolution blocks
        extra_m0_output_channels = None
        if not self.use_s2_act_attn:
            extra_m0_output_channels = self.num_heads * self.attn_alpha_channels
            if self.use_gate_act:
                extra_m0_output_channels = (
                    extra_m0_output_channels
                    + max(self.lmax_list) * self.hidden_channels
                )
            else:
                if self.use_sep_s2_act:
                    extra_m0_output_channels = (
                        extra_m0_output_channels + self.hidden_channels
                    )

        if self.use_m_share_rad:
            self.edge_channels_list = [
                *self.edge_channels_list,
                2 * self.sphere_channels * (max(self.lmax_list) + 1),
            ]
            self.rad_func = RadialFunction(self.edge_channels_list)
            expand_index = torch.zeros([(max(self.lmax_list) + 1) ** 2]).long()
            for lval in range(max(self.lmax_list) + 1):
                start_idx = lval**2
                length = 2 * lval + 1
                expand_index[start_idx : (start_idx + length)] = lval
            self.register_buffer("expand_index", expand_index)

        self.so2_conv_1 = SO2_Convolution(
            2 * self.sphere_channels,
            self.hidden_channels,
            self.lmax_list,
            self.mmax_list,
            self.mappingReduced,
            internal_weights=(bool(self.use_m_share_rad)),
            edge_channels_list=(
                self.edge_channels_list if not self.use_m_share_rad else None
            ),
            extra_m0_output_channels=extra_m0_output_channels,  # for attention weights and/or gate activation
        )

        if self.use_s2_act_attn:
            self.alpha_norm = None
            self.alpha_act = None
            self.alpha_dot = None
        else:
            if self.use_attn_renorm:
                self.alpha_norm = torch.nn.LayerNorm(self.attn_alpha_channels)
            else:
                self.alpha_norm = torch.nn.Identity()
            self.alpha_act = SmoothLeakyReLU()
            self.alpha_dot = torch.nn.Parameter(
                torch.randn(self.num_heads, self.attn_alpha_channels)
            )
            # torch_geometric.nn.inits.glorot(self.alpha_dot) # Following GATv2
            std = 1.0 / math.sqrt(self.attn_alpha_channels)
            torch.nn.init.uniform_(self.alpha_dot, -std, std)

        self.alpha_dropout = None
        if alpha_drop != 0.0:
            self.alpha_dropout = torch.nn.Dropout(alpha_drop)

        if self.use_gate_act:
            self.gate_act = GateActivation(
                lmax=max(self.lmax_list),
                mmax=max(self.mmax_list),
                num_channels=self.hidden_channels,
            )
        else:
            if self.use_sep_s2_act:
                # separable S2 activation
                self.s2_act = SeparableS2Activation(
                    lmax=max(self.lmax_list), mmax=max(self.mmax_list)
                )
            else:
                # S2 activation
                self.s2_act = S2Activation(
                    lmax=max(self.lmax_list), mmax=max(self.mmax_list)
                )

        self.so2_conv_2 = SO2_Convolution(
            self.hidden_channels,
            self.num_heads * self.attn_value_channels,
            self.lmax_list,
            self.mmax_list,
            self.mappingReduced,
            internal_weights=True,
            edge_channels_list=None,
            extra_m0_output_channels=(
                self.num_heads if self.use_s2_act_attn else None
            ),  # for attention weights
        )

        self.proj = SO3_LinearV2(
            self.num_heads * self.attn_value_channels,
            self.output_channels,
            lmax=self.lmax_list[0],
        )

    def forward(
        self,
        x: torch.Tensor,
        edge_distance: torch.Tensor,
        edge_index,
        feature_dict: Dict[str, torch.Tensor] = None,
        node_offset: int = 0,
    ):
        """
        Forward pass through SO2EquivariantGraphAttention.
        
        Args:
            x: SO3_Embedding node features
            edge_distance: [E, radial_basis_size] radial basis encoded distances
            edge_index: [2, E] edge connectivity
            feature_dict: Dictionary of additional node/edge features for EdgeProjector
            node_offset: Node offset for distributed computing
            
        Returns:
            SO3_Embedding: Updated node embeddings
        """
        # Compute edge scalar features (invariant to rotations)
        if self.use_edge_information:
            if feature_dict is None:
                feature_dict = {}
            x_edge = self.edge_projector(edge_distance, edge_index, feature_dict)
        else:
            x_edge = edge_distance

        x_source = x.clone()
        x_target = x.clone()
        if gp_utils.initialized():
            x_full = gp_utils.gather_from_model_parallel_region(x.embedding, dim=0)
            x_source.set_embedding(x_full)
            x_target.set_embedding(x_full)
        x_source._expand_edge(edge_index[:,0])
        x_target._expand_edge(edge_index[:,1])

        x_message_data = torch.cat((x_source.embedding, x_target.embedding), dim=2)
        x_message = SO3_Embedding(
            0,
            x_target.lmax_list.copy(),
            x_target.num_channels * 2,
            device=x_target.device,
            dtype=x_target.dtype,
        )
        x_message.set_embedding(x_message_data)
        x_message.set_lmax_mmax(self.lmax_list.copy(), self.mmax_list.copy())

        # radial function (scale all m components within a type-L vector of one channel with the same weight)
        if self.use_m_share_rad:
            x_edge_weight = self.rad_func(x_edge)
            x_edge_weight = x_edge_weight.reshape(
                -1, (max(self.lmax_list) + 1), 2 * self.sphere_channels
            )
            x_edge_weight = torch.index_select(
                x_edge_weight, dim=1, index=self.expand_index
            )  # [E, (L_max + 1) ** 2, C]
            x_message.embedding = x_message.embedding * x_edge_weight

        # Rotate the irreps to align with the edge
        x_message._rotate(self.SO3_rotation, self.lmax_list, self.mmax_list)

        # First SO(2)-convolution
        if self.use_s2_act_attn:
            x_message = self.so2_conv_1(x_message, x_edge)
        else:
            x_message, x_0_extra = self.so2_conv_1(x_message, x_edge)

        # Activation
        x_alpha_num_channels = self.num_heads * self.attn_alpha_channels
        if self.use_gate_act:
            # Gate activation
            x_0_gating = x_0_extra.narrow(
                1,
                x_alpha_num_channels,
                x_0_extra.shape[1] - x_alpha_num_channels,
            )  # for activation
            x_0_alpha = x_0_extra.narrow(
                1, 0, x_alpha_num_channels
            )  # for attention weights
            x_message.embedding = self.gate_act(x_0_gating, x_message.embedding)
        else:
            if self.use_sep_s2_act:
                x_0_gating = x_0_extra.narrow(
                    1,
                    x_alpha_num_channels,
                    x_0_extra.shape[1] - x_alpha_num_channels,
                )  # for activation
                x_0_alpha = x_0_extra.narrow(
                    1, 0, x_alpha_num_channels
                )  # for attention weights
                x_message.embedding = self.s2_act(
                    x_0_gating, x_message.embedding, self.SO3_grid
                )
            else:
                x_0_alpha = x_0_extra
                x_message.embedding = self.s2_act(x_message.embedding, self.SO3_grid)

        # Second SO(2)-convolution
        if self.use_s2_act_attn:
            x_message, x_0_extra = self.so2_conv_2(x_message, x_edge)
        else:
            x_message = self.so2_conv_2(x_message, x_edge)

        # Attention weights
        if self.use_s2_act_attn:
            alpha = x_0_extra
        else:
            x_0_alpha = x_0_alpha.reshape(-1, self.num_heads, self.attn_alpha_channels)
            x_0_alpha = self.alpha_norm(x_0_alpha)
            x_0_alpha = self.alpha_act(x_0_alpha)
            alpha = torch.einsum("bik, ik -> bi", x_0_alpha, self.alpha_dot)
        alpha = torch_geometric.utils.softmax(alpha, edge_index[:,1])
        alpha = alpha.reshape(alpha.shape[0], 1, self.num_heads, 1)
        if self.alpha_dropout is not None:
            alpha = self.alpha_dropout(alpha)

        # Attention weights * non-linear messages
        attn = x_message.embedding
        attn = attn.reshape(
            attn.shape[0],
            attn.shape[1],
            self.num_heads,
            self.attn_value_channels,
        )
        attn = attn * alpha
        attn = attn.reshape(
            attn.shape[0],
            attn.shape[1],
            self.num_heads * self.attn_value_channels,
        )
        x_message.embedding = attn

        # Rotate back the irreps
        x_message._rotate_inv(self.SO3_rotation, self.mappingReduced)

        # Compute the sum of the incoming neighboring messages for each target node
        x_message._reduce_edge(edge_index[:,1] - node_offset, len(x.embedding))

        # Project
        return self.proj(x_message)

In [95]:
attn = SO2EquivariantGraphAttentionWEdgesV2(
    sphere_channels=NUM_CHANNELS,
    hidden_channels=NUM_CHANNELS,
    num_heads=4,
    attn_alpha_channels=NUM_CHANNELS,
    attn_value_channels=NUM_CHANNELS,
    output_channels=NUM_CHANNELS,
    lmax_list=LMAX_LIST,
    mmax_list=MMAX_LIST,
    SO3_rotation=SO3_rotation,
    mappingReduced=mappingReduced,
    SO3_grid=SO3_grid,
    edge_channels_list=[R.shape[1], NUM_CHANNELS, NUM_CHANNELS],
    use_m_share_rad=False,
    use_edge_information=True,
    radial_basis_size=R.shape[1],
    feature_vocab_sizes=feature_vocab_sizes,
    use_edge_features=True,
    bond_features=collator.featurizer.bond_features,
    use_node_features=True,
    node_features=collator.featurizer.atom_features,
    embedding_dim=32,
    embedding_use_bias=True,
)



In [96]:
message = attn(
    x=h_a,
    edge_distance=R,
    edge_index=batch.edge_index,
    feature_dict=feature_dict,
    node_offset=0
)

In [97]:
message

Wrapped into a full transformer layer with normalization

In [98]:
from fairchem.core.models.equiformer_v2.activation import (
    GateActivation,
    S2Activation,
    SeparableS2Activation,
    SmoothLeakyReLU,
)
from fairchem.core.models.equiformer_v2.drop import EquivariantDropoutArraySphericalHarmonics, GraphDropPath
from fairchem.core.models.equiformer_v2.layer_norm import get_normalization_layer
from fairchem.core.models.equiformer_v2.radial_function import RadialFunction
from fairchem.core.models.equiformer_v2.so2_ops import SO2_Convolution
from fairchem.core.models.equiformer_v2.so3 import SO3_Embedding, SO3_LinearV2
from fairchem.core.models.equiformer_v2.transformer_block import FeedForwardNetwork


class TransBlockV2WithEdges(torch.nn.Module):
    """
    Updated TransBlockV2 that leverages SO2EquivariantGraphAttentionWEdgesV2 
    for enhanced edge information processing.

    Args:
        sphere_channels (int): Number of spherical channels
        attn_hidden_channels (int): Number of hidden channels used during SO(2) graph attention
        num_heads (int): Number of attention heads
        attn_alpha_channels (int): Number of channels for alpha vector in each attention head
        attn_value_channels (int): Number of channels for value vector in each attention head
        ffn_hidden_channels (int): Number of hidden channels used during feedforward network
        output_channels (int): Number of output channels

        lmax_list (list[int]): List of degrees (l) for each resolution
        mmax_list (list[int]): List of orders (m) for each resolution

        SO3_rotation: Class to calculate Wigner-D matrices and rotate embeddings
        mappingReduced: Class to convert l and m indices once node embedding is rotated
        SO3_grid: Class used to convert from grid the spherical harmonic representations

        edge_channels_list (list[int]): List of sizes of invariant edge embedding
        use_m_share_rad (bool): Whether all m components within a type-L vector share radial weights

        # EdgeProjector parameters for enhanced edge information
        use_edge_information (bool): Whether to use edge information in the attention mechanism
        radial_basis_size (int): Number of radial basis functions expected
        feature_vocab_sizes (Dict[str, int]): Dictionary mapping feature names to vocab sizes
        use_edge_features (bool): Whether to use edge features
        bond_features (List[str]): List of bond feature names to use if using any
        use_node_features (bool): Whether to use node features in edge projector
        node_features (List[str]): List of node feature names to use if using any
        embedding_dim (int): Embedding dimension for node and edge features
        embedding_use_bias (bool): Whether to use bias in the embedding layers

        attn_activation (str): Type of activation function for SO(2) graph attention
        use_s2_act_attn (bool): Whether to use attention after S2 activation
        use_attn_renorm (bool): Whether to re-normalize attention weights
        ffn_activation (str): Type of activation function for feedforward network
        use_gate_act (bool): If `True`, use gate activation. Otherwise, use S2 activation
        use_grid_mlp (bool): If `True`, use projecting to grids and performing MLPs for FFN
        use_sep_s2_act (bool): If `True`, use separable S2 activation when `use_gate_act` is False

        norm_type (str): Type of normalization layer (['layer_norm', 'layer_norm_sh'])

        alpha_drop (float): Dropout rate for attention weights
        drop_path_rate (float): Drop path rate
        proj_drop (float): Dropout rate for outputs of attention and FFN
    """

    def __init__(
        self,
        sphere_channels: int,
        attn_hidden_channels: int,
        num_heads: int,
        attn_alpha_channels: int,
        attn_value_channels: int,
        ffn_hidden_channels: int,
        output_channels: int,
        lmax_list: list[int],
        mmax_list: list[int],
        SO3_rotation,
        mappingReduced,
        SO3_grid,
        edge_channels_list: list[int],
        use_m_share_rad: bool = False,
        # EdgeProjector parameters
        use_edge_information: bool = True,
        radial_basis_size: int = 50,
        feature_vocab_sizes: Dict[str, int] = None,
        use_edge_features: bool = True,
        bond_features: List[str] = None,
        use_node_features: bool = True,
        node_features: List[str] = None,
        embedding_dim: int = 32,
        embedding_use_bias: bool = True,
        # Other parameters matching original TransBlockV2
        attn_activation: str = "silu",
        use_s2_act_attn: bool = False,
        use_attn_renorm: bool = True,
        ffn_activation: str = "silu",
        use_gate_act: bool = False,
        use_grid_mlp: bool = False,
        use_sep_s2_act: bool = True,
        norm_type: str = "rms_norm_sh",
        alpha_drop: float = 0.0,
        drop_path_rate: float = 0.0,
        proj_drop: float = 0.0,
    ) -> None:
        super().__init__()

        # Set defaults for edge projector parameters
        if feature_vocab_sizes is None:
            feature_vocab_sizes = {}
        if bond_features is None:
            bond_features = ['bond_order', 'is_in_ring', 'is_aromatic']
        if node_features is None:
            node_features = ['element', 'charge', 'nhyd', 'hyb']

        max_lmax = max(lmax_list)
        
        # Pre-attention normalization
        self.norm_1 = get_normalization_layer(
            norm_type, lmax=max_lmax, num_channels=sphere_channels
        )

        # Enhanced SO2 Graph Attention with edge information
        self.ga = SO2EquivariantGraphAttentionWEdgesV2(
            sphere_channels=sphere_channels,
            hidden_channels=attn_hidden_channels,
            num_heads=num_heads,
            attn_alpha_channels=attn_alpha_channels,
            attn_value_channels=attn_value_channels,
            output_channels=sphere_channels,
            lmax_list=lmax_list,
            mmax_list=mmax_list,
            SO3_rotation=SO3_rotation,
            mappingReduced=mappingReduced,
            SO3_grid=SO3_grid,
            edge_channels_list=edge_channels_list,
            use_m_share_rad=use_m_share_rad,
            # EdgeProjector parameters
            use_edge_information=use_edge_information,
            radial_basis_size=radial_basis_size,
            feature_vocab_sizes=feature_vocab_sizes,
            use_edge_features=use_edge_features,
            bond_features=bond_features,
            use_node_features=use_node_features,
            node_features=node_features,
            embedding_dim=embedding_dim,
            embedding_use_bias=embedding_use_bias,
            # Attention parameters
            activation=attn_activation,
            use_s2_act_attn=use_s2_act_attn,
            use_attn_renorm=use_attn_renorm,
            use_gate_act=use_gate_act,
            use_sep_s2_act=use_sep_s2_act,
            alpha_drop=alpha_drop,
        )

        # Dropout and drop path
        self.drop_path = GraphDropPath(drop_path_rate) if drop_path_rate > 0.0 else None
        self.proj_drop = (
            EquivariantDropoutArraySphericalHarmonics(proj_drop, drop_graph=False)
            if proj_drop > 0.0
            else None
        )

        # Pre-FFN normalization
        self.norm_2 = get_normalization_layer(
            norm_type, lmax=max_lmax, num_channels=sphere_channels
        )

        # Feed-forward network
        self.ffn = FeedForwardNetwork(
            sphere_channels=sphere_channels,
            hidden_channels=ffn_hidden_channels,
            output_channels=output_channels,
            lmax_list=lmax_list,
            mmax_list=mmax_list,
            SO3_grid=SO3_grid,
            activation=ffn_activation,
            use_gate_act=use_gate_act,
            use_grid_mlp=use_grid_mlp,
            use_sep_s2_act=use_sep_s2_act,
        )

        # Optional shortcut connection for dimension mismatch
        if sphere_channels != output_channels:
            self.ffn_shortcut = SO3_LinearV2(
                sphere_channels, output_channels, lmax=max_lmax
            )
        else:
            self.ffn_shortcut = None

    def forward(
        self,
        x,  # SO3_Embedding
        edge_distance: torch.Tensor,
        edge_index: torch.Tensor,
        feature_dict: Dict[str, torch.Tensor],
        batch=None,  # for GraphDropPath
        node_offset: int = 0,
    ):
        """
        Forward pass through the transformer block.
        
        Args:
            x: SO3_Embedding node features
            edge_distance: [E, radial_basis_size] radial basis encoded distances
            edge_index: [2, E] edge connectivity
            feature_dict: Dictionary of additional node/edge features for EdgeProjector
            batch: Batch information for GraphDropPath
            node_offset: Node offset for distributed computing
            
        Returns:
            SO3_Embedding: Updated node embeddings
        """
        output_embedding = x.clone()

        # Store residual connection for attention
        x_res = output_embedding.clone()

        # Pre-attention normalization
        output_embedding.embedding = self.norm_1(output_embedding.embedding)

        # Enhanced attention with edge information
        output_embedding = self.ga(
            x=output_embedding,
            edge_distance=edge_distance,
            edge_index=edge_index,
            feature_dict=feature_dict,
            node_offset=node_offset,
        )

        # Apply projection dropout if specified
        if self.proj_drop is not None:
            output_embedding.embedding = self.proj_drop(output_embedding.embedding)

        # Apply drop path if specified
        if self.drop_path is not None:
            output_embedding.embedding = self.drop_path(
                output_embedding.embedding, batch
            )

        # Residual connection for attention
        output_embedding.embedding = output_embedding.embedding + x_res.embedding

        # Store residual connection for FFN
        x_res = output_embedding.clone()

        # Pre-FFN normalization
        output_embedding.embedding = self.norm_2(output_embedding.embedding)

        # Feed-forward network
        output_embedding = self.ffn(output_embedding)

        # Apply projection dropout if specified
        if self.proj_drop is not None:
            output_embedding.embedding = self.proj_drop(output_embedding.embedding)

        # Apply drop path if specified
        if self.drop_path is not None:
            output_embedding.embedding = self.drop_path(
                output_embedding.embedding, batch
            )

        # Residual connection for FFN (with optional shortcut for dimension mismatch)
        if self.ffn_shortcut is not None:
            x_res = self.ffn_shortcut(x_res)
        output_embedding.embedding = output_embedding.embedding + x_res.embedding

        return output_embedding

In [99]:
trans = TransBlockV2WithEdges(
    sphere_channels=NUM_CHANNELS,
    attn_hidden_channels=NUM_CHANNELS,
    num_heads=4,
    attn_alpha_channels=NUM_CHANNELS,
    attn_value_channels=NUM_CHANNELS,
    ffn_hidden_channels=NUM_CHANNELS,
    output_channels=NUM_CHANNELS,
    lmax_list=LMAX_LIST,
    mmax_list=MMAX_LIST,
    SO3_rotation=SO3_rotation,
    mappingReduced=mappingReduced,
    SO3_grid=SO3_grid,
    edge_channels_list=[R.shape[1], NUM_CHANNELS, NUM_CHANNELS],
    use_m_share_rad=False,
    feature_vocab_sizes=feature_vocab_sizes,
    use_edge_information=True, radial_basis_size=R.shape[1],)
    

In [100]:
h_a = trans(
    x=h_a,
    edge_distance=R,
    edge_index=batch.edge_index,
    feature_dict=feature_dict,
    node_offset=0
)

### Whole backbone class

In [ ]:
import contextlib
import logging
import typing
from functools import partial
from typing import Dict, List, Optional

import torch
import torch.nn as nn

from fairchem.core.common import gp_utils
from fairchem.core.common.registry import registry
from fairchem.core.common.utils import conditional_grad
from fairchem.core.models.base import GraphModelMixin
from fairchem.core.models.scn.smearing import GaussianSmearing

from fairchem.core.models.equiformer_v2.edge_rot_mat import init_edge_rot_mat
from fairchem.core.models.equiformer_v2.gaussian_rbf import GaussianRadialBasisLayer
from fairchem.core.models.equiformer_v2.layer_norm import get_normalization_layer
from fairchem.core.models.equiformer_v2.module_list import ModuleListInfo
from fairchem.core.models.equiformer_v2.so3 import (
    CoefficientMappingModule,
    SO3_Embedding,
    SO3_Grid,
    SO3_LinearV2,
    SO3_Rotation,
)
from fairchem.core.models.equiformer_v2.weight_initialization import eqv2_init_weights

# Import custom modules from notebook (assumed to be available)
# These would normally be imported from their respective modules
# from .custom_modules import (
#     NodeEmbedder,
#     EdgeEmbedder, 
#     SO3ScalarEmbedder,
#     EdgeDegreeEmbedding,
#     SO3_L1_LinearMixing,
#     TransBlockV2WithEdges,
#     compute_positional_topology_gradients
# )

with contextlib.suppress(ImportError):
    pass

if typing.TYPE_CHECKING:
    from metalsitenn.dataloading import BatchProteinData

# Statistics of IS2RE 100K (same as original)
_AVG_NUM_NODES = 77.81317
_AVG_DEGREE = 23.395238876342773


# @registry.register_model("equiformer_w_edges_backbone")
class EquiformerWEdgesBackbone(nn.Module):
    """
    Enhanced Equiformer backbone with molecular feature support and edge information processing.
    
    Designed to accept BatchProteinData with:
    - Precomputed molecular graphs with rich atom/bond features
    - Topology information for L1 gradient computation
    - Enhanced edge information processing in attention layers
    
    Args:
        num_layers (int): Number of transformer layers
        sphere_channels (int): Number of spherical channels (one set per resolution)
        attn_hidden_channels (int): Number of hidden channels used during SO(2) graph attention
        num_heads (int): Number of attention heads
        attn_alpha_channels (int): Number of channels for alpha vector in each attention head
        attn_value_channels (int): Number of channels for value vector in each attention head
        ffn_hidden_channels (int): Number of hidden channels used during feedforward network
        norm_type (str): Type of normalization layer (['layer_norm', 'layer_norm_sh', 'rms_norm_sh'])
        lmax_list (list[int]): List of degrees (l) for each resolution
        mmax_list (list[int]): List of orders (m) for each resolution
        grid_resolution (int): Resolution of SO3_grid
        num_sphere_samples (int): Number of samples used to approximate the integration of the sphere in the output blocks
        edge_channels_list (list[int]): List of sizes of invariant edge embedding
        use_m_share_rad (bool): Whether all m components within a type-L vector share radial function weights
        distance_function (str): Type of distance function (['gaussian', 'gaussian_rbf'])
        num_distance_basis (int): Number of distance basis functions
        attn_activation (str): Type of activation function for SO(2) graph attention
        use_s2_act_attn (bool): Whether to use attention after S2 activation
        use_attn_renorm (bool): Whether to re-normalize attention weights
        ffn_activation (str): Type of activation function for feedforward network
        use_gate_act (bool): If `True`, use gate activation. Otherwise, use S2 activation
        use_grid_mlp (bool): If `True`, use projecting to grids and performing MLPs for FFN
        use_sep_s2_act (bool): If `True`, use separable S2 activation when `use_gate_act` is False
        alpha_drop (float): Dropout rate for attention weights
        drop_path_rate (float): Drop path rate
        proj_drop (float): Dropout rate for outputs of attention and FFN
        weight_init (str): Weight initialization method
        
        # Enhanced molecular feature parameters
        feature_vocab_sizes (Dict[str, int]): Vocabulary sizes for molecular features
        atom_features (List[str]): List of atom feature names to use
        bond_features (List[str]): List of bond feature names to use
        embedding_dim (int): Embedding dimension for molecular features
        use_topology_gradients (bool): Whether to use topology gradients for L1 features
        topology_gradient_clip (float): Gradient clipping value for topology gradients
        
        # Legacy parameters (ignored with warnings)
        regress_forces (bool): IGNORED - Forces regression handled by separate head modules
        otf_graph (bool): IGNORED - Uses precomputed graphs from BatchProteinData
        use_pbc (bool): IGNORED - PBC handled in data preprocessing
        use_pbc_single (bool): IGNORED - PBC handled in data preprocessing
        max_neighbors (int): IGNORED - Applied during graph preprocessing
        max_radius (float): Used only for distance_expansion range
        max_num_elements (int): IGNORED - Use feature_vocab_sizes["element"] instead
        use_atom_edge_embedding (bool): IGNORED - Replaced by enhanced molecular features
        share_atom_edge_embedding (bool): IGNORED - Replaced by enhanced molecular features
    """
    
    def __init__(
        self,
        # Core architecture parameters
        num_layers: int = 12,
        sphere_channels: int = 128,
        attn_hidden_channels: int = 128,
        num_heads: int = 8,
        attn_alpha_channels: int = 32,
        attn_value_channels: int = 16,
        ffn_hidden_channels: int = 512,
        norm_type: str = "rms_norm_sh",
        lmax_list: Optional[List[int]] = None,
        mmax_list: Optional[List[int]] = None,
        grid_resolution: Optional[int] = None,
        num_sphere_samples: int = 128,
        edge_channels_list: Optional[List[int]] = None,
        use_m_share_rad: bool = False,
        distance_function: str = "gaussian",
        num_distance_basis: int = 512,
        attn_activation: str = "scaled_silu",
        use_s2_act_attn: bool = False,
        use_attn_renorm: bool = True,
        ffn_activation: str = "scaled_silu",
        use_gate_act: bool = False,
        use_grid_mlp: bool = False,
        use_sep_s2_act: bool = True,
        alpha_drop: float = 0.0,
        drop_path_rate: float = 0.0,
        proj_drop: float = 0.0,
        weight_init: str = "uniform",
        max_radius: float = 5.0,  # Used for distance_expansion range
        # Enhanced molecular feature parameters
        feature_vocab_sizes: Optional[Dict[str, int]] = None,
        atom_features: Optional[List[str]] = None,
        bond_features: Optional[List[str]] = None,
        embedding_dim: int = 32,
        use_topology_gradients: bool = True,
        topology_gradient_clip: float = 100.0,
        # Legacy parameters (ignored with warnings)
        regress_forces: bool = True,
        otf_graph: bool = True,
        use_pbc: bool = True,
        use_pbc_single: bool = False,
        max_neighbors: int = 500,
        max_num_elements: int = 90,
        use_atom_edge_embedding: bool = True,
        share_atom_edge_embedding: bool = False,
        # previously global params
        avg_num_nodes: float = _AVG_NUM_NODES,
        avg_degree: float = _AVG_DEGREE,

        **kwargs  # Catch any other legacy parameters
    ):
        super().__init__()
        
        # Store core architecture parameters
        self.num_layers = num_layers
        self.sphere_channels = sphere_channels
        self.attn_hidden_channels = attn_hidden_channels
        self.num_heads = num_heads
        self.attn_alpha_channels = attn_alpha_channels
        self.attn_value_channels = attn_value_channels
        self.ffn_hidden_channels = ffn_hidden_channels
        self.norm_type = norm_type
        self.lmax_list = lmax_list or [6]
        self.mmax_list = mmax_list or [2]
        self.grid_resolution = grid_resolution
        self.num_sphere_samples = num_sphere_samples
        self.num_distance_basis = num_distance_basis
        self.edge_channels_list = edge_channels_list or [self.num_distance_basis, self.sphere_channels, self.sphere_channels]
        self.use_m_share_rad = use_m_share_rad
        self.distance_function = distance_function
        self.attn_activation = attn_activation
        self.use_s2_act_attn = use_s2_act_attn
        self.use_attn_renorm = use_attn_renorm
        self.ffn_activation = ffn_activation
        self.use_gate_act = use_gate_act
        self.use_grid_mlp = use_grid_mlp
        self.use_sep_s2_act = use_sep_s2_act
        self.alpha_drop = alpha_drop
        self.drop_path_rate = drop_path_rate
        self.proj_drop = proj_drop
        self.weight_init = weight_init
        self.max_radius = max_radius
        
        # Store enhanced feature parameters
        self.feature_vocab_sizes = feature_vocab_sizes or {}
        self.atom_features = atom_features or ['element', 'charge', 'nhyd', 'hyb']
        self.bond_features = bond_features or ['bond_order', 'is_in_ring', 'is_aromatic']
        self.embedding_dim = embedding_dim
        self.use_topology_gradients = use_topology_gradients
        self.topology_gradient_clip = topology_gradient_clip
        
        # Computed properties (same as original EquiformerV2)
        self.num_resolutions = len(self.lmax_list)
        self.sphere_channels_all = self.num_resolutions * self.sphere_channels
        self.avg_num_nodes = avg_num_nodes
        self.avg_degree = avg_degree
        
        # Warn about ignored parameters
        self._warn_ignored_parameters(
            regress_forces=regress_forces,
            otf_graph=otf_graph,
            use_pbc=use_pbc,
            use_pbc_single=use_pbc_single,
            max_neighbors=max_neighbors,
            max_num_elements=max_num_elements,
            use_atom_edge_embedding=use_atom_edge_embedding,
            share_atom_edge_embedding=share_atom_edge_embedding,
            **kwargs
        )
        
        # Initialize all components
        self._init_so3_components()
        self._init_distance_expansion()
        self._init_enhanced_embeddings()
        self._init_normalization_layers()
        self._init_enhanced_transformer_blocks()
        
        if self.use_topology_gradients:
            self._init_topology_mixing()
        
        # Apply weight initialization
        self.apply(partial(eqv2_init_weights, weight_init=self.weight_init))
    
    def _warn_ignored_parameters(self, **kwargs):
        """Warn about parameters that are ignored in EquiformerWEdgesBackbone."""
        ignored_params = {
            'regress_forces': 'Forces regression is handled by separate head modules',
            'otf_graph': 'On-the-fly graph construction not used (precomputed graphs expected)',
            'use_pbc': 'Periodic boundary conditions handled in data preprocessing',
            'use_pbc_single': 'PBC single processing handled in data preprocessing', 
            'max_neighbors': 'Max neighbors constraint applied during graph preprocessing',
            'max_num_elements': 'Element vocabulary determined by feature_vocab_sizes["element"]',
            'use_atom_edge_embedding': 'Atom-edge embedding replaced by enhanced molecular feature embedding',
            'share_atom_edge_embedding': 'Embedding sharing replaced by enhanced molecular feature embedding',
        }
        
        for param_name, reason in ignored_params.items():
            if param_name in kwargs:
                param_value = kwargs[param_name]
                # Check if parameter has non-default value that might indicate user expects it to work
                default_values = {
                    'regress_forces': True,
                    'otf_graph': True, 
                    'use_pbc': True,
                    'use_pbc_single': False,
                    'max_neighbors': 500,
                    'max_num_elements': 90,
                    'use_atom_edge_embedding': True,
                    'share_atom_edge_embedding': False,
                }
                
                if param_value != default_values.get(param_name):
                    logging.warning(
                        f"Parameter '{param_name}={param_value}' is ignored in EquiformerWEdgesBackbone. "
                        f"Reason: {reason}"
                    )
        
        # Warn about any unknown kwargs
        for param_name in kwargs:
            if param_name not in ignored_params:
                logging.warning(f"Unknown parameter '{param_name}' ignored in EquiformerWEdgesBackbone")

        # set them any way for compatibility
        for param_name in ignored_params:
            setattr(self, param_name, kwargs.get(param_name, None))
    
    def _init_so3_components(self):
        """Initialize SO3 components (same as EquiformerV2Backbone)."""
        # Coefficient mapping
        self.mappingReduced = CoefficientMappingModule(self.lmax_list, self.mmax_list)
        
        # SO3 rotation modules
        self.SO3_rotation = nn.ModuleList()
        for i in range(self.num_resolutions):
            self.SO3_rotation.append(SO3_Rotation(self.lmax_list[i]))
        
        # SO3 grid
        self.SO3_grid = ModuleListInfo(f"({max(self.lmax_list)}, {max(self.lmax_list)})")
        for lval in range(max(self.lmax_list) + 1):
            SO3_m_grid = nn.ModuleList()
            for m in range(max(self.lmax_list) + 1):
                SO3_m_grid.append(
                    SO3_Grid(
                        lval,
                        m,
                        resolution=self.grid_resolution,
                        normalization="component",
                    )
                )
            self.SO3_grid.append(SO3_m_grid)
    
    def _init_distance_expansion(self):
        """Initialize distance expansion module."""
        if self.distance_function == "gaussian":
            self.distance_expansion = GaussianSmearing(
                start=0.0,
                stop=self.max_radius,
                num_gaussians=self.num_distance_basis,
                basis_width_scalar=(self.max_radius - 0.0) / self.num_distance_basis
            )
        elif self.distance_function == "gaussian_rbf":
            self.distance_expansion = GaussianRadialBasisLayer(
                num_basis=self.num_distance_basis,
                cutoff=self.max_radius,
            )
        else:
            raise ValueError(f"Unsupported distance function: {self.distance_function}")
    
    def _init_enhanced_embeddings(self):
        """Initialize enhanced embedding modules for molecular features."""
        
        # Enhanced node embedder for atom features
        self.node_embedder = NodeEmbedder(
            feature_vocab_sizes=self.feature_vocab_sizes,
            atom_features=self.atom_features,
            output_dim=self.sphere_channels_all,
            embedding_dim=self.embedding_dim,
        )
        
        # SO3 scalar embedder to convert node embeddings to SO3 format
        self.so3_scalar_embedder = SO3ScalarEmbedder(
            lmax_list=self.lmax_list,
            sphere_channels=self.sphere_channels,
        )
        
        # Enhanced edge degree embedding with molecular features
        self.edge_deg_embedding = EdgeDegreeEmbedding(
            sphere_channels=self.sphere_channels,
            lmax_list=self.lmax_list,
            mmax_list=self.mmax_list,
            SO3_rotation=self.SO3_rotation,
            mappingReduced=self.mappingReduced,
            radial_basis_size=self.num_distance_basis,
            feature_vocab_sizes=self.feature_vocab_sizes,
            use_edge_features=True,
            bond_features=self.bond_features,
            use_node_features=True,
            node_features=self.atom_features,
            embedding_dim=self.embedding_dim,
            embedding_use_bias=True,
            projector_hidden_layers=2,
            projector_size=64,
            rescale_factor=1.0,
        )
    
    def _init_normalization_layers(self):
        """Initialize normalization layers."""
        self.norm_1 = get_normalization_layer(
            self.norm_type,
            lmax=max(self.lmax_list),
            num_channels=self.sphere_channels_all
        )
        
        self.norm_final = get_normalization_layer(
            self.norm_type,
            lmax=max(self.lmax_list),
            num_channels=self.sphere_channels_all
        )
    
    def _init_enhanced_transformer_blocks(self):
        """Initialize transformer blocks with edge-aware versions."""
        
        self.blocks = nn.ModuleList()
        for i in range(self.num_layers):
            block = TransBlockV2WithEdges(
                sphere_channels=self.sphere_channels,
                attn_hidden_channels=self.attn_hidden_channels,
                num_heads=self.num_heads,
                attn_alpha_channels=self.attn_alpha_channels,
                attn_value_channels=self.attn_value_channels,
                ffn_hidden_channels=self.ffn_hidden_channels,
                output_channels=self.sphere_channels,
                lmax_list=self.lmax_list,
                mmax_list=self.mmax_list,
                SO3_rotation=self.SO3_rotation,
                mappingReduced=self.mappingReduced,
                SO3_grid=self.SO3_grid,
                edge_channels_list=self.edge_channels_list,
                use_m_share_rad=self.use_m_share_rad,
                # Enhanced edge information parameters
                use_edge_information=True,
                radial_basis_size=self.num_distance_basis,
                feature_vocab_sizes=self.feature_vocab_sizes,
                use_edge_features=True,
                bond_features=self.bond_features,
                use_node_features=True,
                node_features=self.atom_features,
                embedding_dim=self.embedding_dim,
                embedding_use_bias=True,
                # Other transformer parameters
                attn_activation=self.attn_activation,
                use_s2_act_attn=self.use_s2_act_attn,
                use_attn_renorm=self.use_attn_renorm,
                ffn_activation=self.ffn_activation,
                use_gate_act=self.use_gate_act,
                use_grid_mlp=self.use_grid_mlp,
                use_sep_s2_act=self.use_sep_s2_act,
                norm_type=self.norm_type,
                alpha_drop=self.alpha_drop,
                drop_path_rate=self.drop_path_rate,
                proj_drop=self.proj_drop,
            )
            self.blocks.append(block)
    
    def _init_topology_mixing(self):
        """Initialize topology gradient mixing for L1 features."""
        self.topology_mixer = SO3_L1_LinearMixing(
            in_channels_list=[self.sphere_channels_all, 3],  # SO3 L1 + topology gradients
            out_channels=self.sphere_channels_all
        )
    
    def _extract_feature_dict(self, data: 'BatchProteinData') -> Dict[str, torch.Tensor]:
        """Extract molecular features from BatchProteinData into dictionary format."""
        feature_dict = {}
        
        # Extract atom features
        for feature in self.atom_features:
            if hasattr(data, feature):
                feature_dict[feature] = getattr(data, feature)
            else:
                raise ValueError(f"Atom feature {feature} not found in batch data")
        
        # Extract bond features  
        for feature in self.bond_features:
            if hasattr(data, feature):
                feature_dict[feature] = getattr(data, feature)
            else:
                raise ValueError(f"Bond feature {feature} not found in batch data")
        
        return feature_dict
    
    def _compute_topology_gradients(
        self, 
        data: 'BatchProteinData'
    ) -> torch.Tensor:
        """Compute topology gradients for L1 features."""
        if not self.use_topology_gradients:
            return None
            
        # Create atom mask for any masked atoms
        masked_elements = None
        # TODO: check for masked token
        
        atom_mask = None if masked_elements is None else ~masked_elements
        
        # Extract topology information
        topology = data.topology if hasattr(data, 'topology') else {}
        bonds = topology.get('bonds', torch.empty(0, 2, dtype=torch.long, device=data.positions.device))
        bond_lengths = topology.get('bond_lengths', torch.empty(0, 1, device=data.positions.device))
        chirals = topology.get('chirals', torch.empty(0, 4, dtype=torch.long, device=data.positions.device))
        planars = topology.get('planars', torch.empty(0, 4, dtype=torch.long, device=data.positions.device))
        
        # Compute gradients
        gradients = compute_positional_topology_gradients(
            r=data.positions,
            bond_indexes=bonds,
            bond_lengths=bond_lengths,
            chirals=chirals,
            planars=planars,
            gclip=self.topology_gradient_clip,
            atom_mask=atom_mask
        )
        
        return gradients
    
    # @conditional_grad(torch.enable_grad()) This thing is messing us up thinking we might want to compute gradients of energy to get forces - we won't
    def forward(self, data: 'BatchProteinData'):
        """
        Forward pass with enhanced molecular feature processing.
        
        Args:
            data: BatchProteinData containing precomputed graphs and molecular features
            
        Returns:
            dict: Dictionary containing node embeddings and graph information
        """
        atomic_numbers = data.element
        num_atoms = len(atomic_numbers)
        
        # Extract molecular features
        feature_dict = self._extract_feature_dict(data)
        
        # Compute edge rotation matrices for SO3 operations
        edge_index = data.edge_index
        edge_distance = data.distances
        edge_distance_vec = data.distance_vec
        edge_rot_mat = init_edge_rot_mat(edge_distance_vec)
        
        # Set up SO3 rotation matrices
        for i, SO3_rot in enumerate(self.SO3_rotation):
            SO3_rot.set_wigner(edge_rot_mat)
        
        # Distance embedding using enhanced radial basis
        edge_distance_rbf = self.distance_expansion(edge_distance)
        
        ###############################################################
        # Initialize node embeddings
        ###############################################################
        
        # Enhanced node embedding with molecular features
        node_attributes = self.node_embedder(feature_dict)
        
        # Convert to SO3 embedding format (L=0 coefficients)
        node_embedding = self.so3_scalar_embedder(node_attributes)
        
        ###############################################################
        # Add edge information to node embeddings
        ###############################################################
        
        # Enhanced edge degree embedding with molecular features
        edge_embedding = self.edge_deg_embedding(
            edge_distance_rbf=edge_distance_rbf,
            edge_index=edge_index,
            num_nodes=num_atoms,
            feature_dict=feature_dict,
            node_offset=0
        )
        
        # Add edge information to node embeddings
        node_embedding.embedding = node_embedding.embedding + edge_embedding.embedding
        
        # Apply input normalization
        node_embedding.embedding = self.norm_1(node_embedding.embedding)
        
        ###############################################################
        # Mix topology gradients into L1 features (if enabled)
        ###############################################################
        
        if self.use_topology_gradients:
            topology_gradients = self._compute_topology_gradients(data)
            if topology_gradients is not None:
                # Extract current L1 features
                current_l1_features = node_embedding.embedding[:, 1:4, :]  # L=1, m=-1,0,1
                
                # Mix with topology gradients
                mixed_l1_features = self.topology_mixer([current_l1_features, topology_gradients])
                
                # Update L1 features in embedding
                node_embedding.embedding[:, 1:4, :] = mixed_l1_features
        
        ###############################################################
        # Transformer blocks with enhanced edge information
        ###############################################################
        
        for i in range(self.num_layers):
            node_embedding = self.blocks[i](
                x=node_embedding,
                edge_distance=edge_distance_rbf,
                edge_index=edge_index,
                feature_dict=feature_dict,
                batch=data.batch if hasattr(data, 'batch') else None,
                node_offset=0,
            )
        
        # Final output normalization
        node_embedding.embedding = self.norm_final(node_embedding.embedding)
        
        ###############################################################
        # Return embeddings in format compatible with heads
        ###############################################################
        
        # Create graph information object (similar to original)
        graph_info = type('GraphInfo', (), {
            'atomic_numbers': atomic_numbers,
            'atomic_numbers_full': atomic_numbers,  # For compatibility
            'edge_distance': edge_distance_rbf,
            'edge_index': edge_index,
            'node_offset': 0,
        })()
        
        return {
            "node_embedding": node_embedding,
            "graph": graph_info,
        }

In [102]:
backbone = EquiformerWEdgesBackbone(
    feature_vocab_sizes=feature_vocab_sizes,
    atom_features=['element', 'charge', 'nhyd', 'hyb'],
    bond_features=['bond_order', 'is_in_ring', 'is_aromatic'],
    embedding_dim=32,
    use_topology_gradients=True,
)

In [103]:
backbone

EquiformerWEdgesBackbone(
  (mappingReduced): CoefficientMappingModule(lmax_list=[6], mmax_list=[2])
  (SO3_rotation): ModuleList(
    (0): SO3_Rotation(
      (mapping): CoefficientMappingModule(lmax_list=[6], mmax_list=[6])
    )
  )
  (SO3_grid): (6, 6)
  (distance_expansion): GaussianSmearing()
  (node_embedder): NodeEmbedder(
    (embeddings): ModuleDict(
      (element): Embedding(46, 32)
      (charge): Embedding(8, 32)
      (nhyd): Embedding(7, 32)
      (hyb): Embedding(8, 32)
    )
    (projection): Linear(in_features=128, out_features=128, bias=True)
  )
  (so3_scalar_embedder): SO3ScalarEmbedder()
  (edge_deg_embedding): EdgeDegreeEmbedding(
    (SO3_rotation): ModuleList(
      (0): SO3_Rotation(
        (mapping): CoefficientMappingModule(lmax_list=[6], mmax_list=[6])
      )
    )
    (mappingReduced): CoefficientMappingModule(lmax_list=[6], mmax_list=[2])
    (rad_func): EdgeProjector(
      (source_embedding): NodeEmbedder(
        (embeddings): ModuleDict(
          

In [104]:
outs = backbone(batch)

In [105]:
# count params
def count_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {count_params(backbone)}")

Total trainable parameters: 130982208


### Time input using film?

In [ ]:
import math
import torch
import torch.nn as nn
from typing import List, Union, Optional
from fairchem.core.models.scn.smearing import GaussianSmearing
from fairchem.core.models.equiformer_v2.so3 import CoefficientMappingModule

from metalsitenn.nn.mlp import MLP


class SO3EquivariantFiLM(nn.Module):
    """
    SO(3)-equivariant FiLM modulation using CoefficientMappingModule for proper indexing.
    
    Args:
        lmax_list: List of maximum degrees for each resolution
        num_channels: Number of spherical channels
        num_layers: Number of model layers (e.g., attention layers)
        mmax_list: List of maximum orders for each resolution (defaults to lmax_list)
        time_embedding_dim: Dimension for time embedding after gaussian smearing
        hidden_dim: Hidden dimension for MLP
        mlp_layers: Number of layers in time embedding MLP
        num_gaussians: Number of gaussian basis functions for time smearing
        sigma_min: Minimum sigma for gaussian smearing (default: 1e-4)
        sigma_max: Maximum sigma for gaussian smearing (default: 1.0)
        
    Returns:
        Tensor of shape (batch_size, num_coefficients, num_channels) for single layer
        OR (num_layers, batch_size, num_coefficients, num_channels) for multi-layer
    """
    
    def __init__(
        self,
        lmax_list: List[int],
        num_channels: int,
        num_layers: int = 1,  # Default to single layer
        mmax_list: List[int] = None,
        time_embedding_dim: int = 128,
        hidden_dim: int = 256,
        mlp_layers: int = 3,
        num_gaussians: int = 50,
        sigma_min: float = 1e-4,
        sigma_max: float = 1.0,
    ):
        super().__init__()
        
        self.lmax_list = lmax_list
        self.mmax_list = mmax_list if mmax_list is not None else lmax_list.copy()
        self.num_channels = num_channels
        self.num_layers = num_layers
        self.num_resolutions = len(lmax_list)
        
        # Use Equiformer's coefficient mapping for exact compatibility
        self.mapping = CoefficientMappingModule(self.lmax_list, self.mmax_list)
        
        # Gaussian time embedding
        self.time_embedding = GaussianSmearing(
            start=sigma_min,
            stop=sigma_max, 
            num_gaussians=num_gaussians
        )
        
        # Time encoding MLP
        self.time_mlp = MLP(
            input_size=num_gaussians,
            hidden_size=hidden_dim,
            n_hidden_layers=mlp_layers - 1,
            hidden_activation='silu',
            output_activation=None,
            dropout_rate=0.0
        )
        
        # Final projection to time_embedding_dim if different from hidden_dim
        if hidden_dim != time_embedding_dim:
            self.time_proj = nn.Linear(hidden_dim, time_embedding_dim, bias=False)
        else:
            self.time_proj = nn.Identity()
        
        # Compute coefficient structure using Equiformer's mapping
        self._compute_coefficient_structure()
        
        # Projection for gamma parameters
        # For each (resolution, l), we need num_channels gamma values
        self.total_l_channels = sum(
            num_channels * (self.lmax_list[res_idx] + 1) 
            for res_idx in range(self.num_resolutions)
        )
        
        if self.num_layers == 1:
            # Single layer output
            self.film_projection = nn.Linear(
                time_embedding_dim, 
                self.total_l_channels, 
                bias=False
            )
        else:
            # Multi-layer output
            self.film_projection = nn.Linear(
                time_embedding_dim, 
                self.num_layers * self.total_l_channels, 
                bias=False
            )
    
    def _compute_coefficient_structure(self):
        """Compute coefficient structure using Equiformer's CoefficientMappingModule."""
        # Get total number of coefficients from the mapping module
        with torch.no_grad():
            # Create dummy tensors to determine structure
            device = next(self.parameters()).device if len(list(self.parameters())) > 0 else 'cpu'
            self.mapping.device = device
            
            # Get l and m harmonics from the mapping
            l_harmonic = self.mapping.l_harmonic.to(device)
            m_harmonic = self.mapping.m_harmonic.to(device)
            
            self.total_coefficients = len(l_harmonic)
            
            # Create mapping from coefficient index to (resolution, l, m)
            self.coeff_to_res_l_m = []
            self.res_l_to_coeff_indices = {}
            self.res_l_to_proj_slice = {}
            
            # Track where each resolution starts
            offset = 0
            proj_offset = 0
            
            for res_idx in range(self.num_resolutions):
                lmax = self.lmax_list[res_idx]
                mmax = self.mmax_list[res_idx]
                
                res_coeffs = 0
                for l in range(lmax + 1):
                    # Count coefficients for this l in this resolution
                    actual_mmax = min(mmax, l)
                    num_m = 2 * actual_mmax + 1
                    
                    # Find coefficient indices for this (res, l)
                    coeff_indices = []
                    for m_idx in range(-actual_mmax, actual_mmax + 1):
                        coeff_indices.append(offset + res_coeffs)
                        self.coeff_to_res_l_m.append((res_idx, l, m_idx))
                        res_coeffs += 1
                    
                    self.res_l_to_coeff_indices[(res_idx, l)] = coeff_indices
                    
                    # Projection slice for this (res, l)
                    self.res_l_to_proj_slice[(res_idx, l)] = slice(
                        proj_offset, proj_offset + self.num_channels
                    )
                    proj_offset += self.num_channels
                
                offset += res_coeffs
    
    def forward(self, time: torch.Tensor) -> torch.Tensor:
        """
        Forward pass of SO(3)-equivariant FiLM.
        
        Args:
            time: Time values of shape (batch_size,) or (batch_size, 1)
        
        Returns:
            gamma: Modulation tensor 
                   Shape (batch_size, num_coefficients, num_channels) if num_layers=1
                   Shape (num_layers, batch_size, num_coefficients, num_channels) if num_layers>1
        """
        batch_size = time.shape[0]
        if time.dim() == 1:
            time = time.unsqueeze(-1)
        
        # a) Gaussian smearing of time
        time_embedded = self.time_embedding(time)  # (batch_size, num_gaussians)
        
        # b) Pass through MLP  
        time_features = self.time_mlp(time_embedded)  # (batch_size, hidden_dim)
        time_features = self.time_proj(time_features)  # (batch_size, time_embedding_dim)
        
        # c) Project to gamma parameters
        all_gammas = self.film_projection(time_features)  # (batch_size, total_l_channels * num_layers)
        
        if self.num_layers == 1:
            return self._expand_single_layer(all_gammas, batch_size)
        else:
            return self._expand_multi_layer(all_gammas, batch_size)
    
    def _expand_single_layer(self, gammas: torch.Tensor, batch_size: int) -> torch.Tensor:
        """Expand gammas for single layer output."""
        device = gammas.device
        dtype = gammas.dtype
        
        # Initialize output tensor
        gamma_output = torch.zeros(
            (batch_size, self.total_coefficients, self.num_channels),
            dtype=dtype, device=device
        )
        
        # Fill in each (res, l) combination
        for res_idx in range(self.num_resolutions):
            lmax = self.lmax_list[res_idx]
            for l in range(lmax + 1):
                # Extract gamma for this (res, l)
                proj_slice = self.res_l_to_proj_slice[(res_idx, l)]
                l_gamma = gammas[:, proj_slice]  # (batch_size, num_channels)
                
                # Get coefficient indices for this (res, l)
                coeff_indices = self.res_l_to_coeff_indices[(res_idx, l)]
                
                # Broadcast: same gamma for all m values within this (res, l)
                for coeff_idx in coeff_indices:
                    gamma_output[:, coeff_idx, :] = l_gamma
        
        return gamma_output
    
    def _expand_multi_layer(self, all_gammas: torch.Tensor, batch_size: int) -> torch.Tensor:
        """Expand gammas for multi-layer output."""
        # Reshape to separate layers
        all_gammas = all_gammas.view(
            batch_size, self.num_layers, self.total_l_channels
        )  # (batch_size, num_layers, total_l_channels)
        
        layer_gammas = []
        for layer_idx in range(self.num_layers):
            layer_gamma = all_gammas[:, layer_idx, :]  # (batch_size, total_l_channels)
            expanded_gamma = self._expand_single_layer(layer_gamma, batch_size)
            layer_gammas.append(expanded_gamma)
        
        # Stack to get shape (num_layers, batch_size, num_coefficients, num_channels)
        return torch.stack(layer_gammas, dim=0)
    
    def get_output_dim(self) -> int:
        """Return the total number of coefficients for validation."""
        return self.total_coefficients
        
    def extra_repr(self) -> str:
        return (f"lmax_list={self.lmax_list}, mmax_list={self.mmax_list}, "
                f"num_channels={self.num_channels}, num_layers={self.num_layers}, "
                f"total_coefficients={self.total_coefficients}")

In [173]:
film = SO3EquivariantFiLM(
    lmax_list=LMAX_LIST,
    mmax_list=MMAX_LIST,
    num_channels=NUM_CHANNELS,
    time_embedding_dim=NUM_CHANNELS,
    hidden_dim=NUM_CHANNELS,
    num_layers=3,
    num_gaussians=50,
    sigma_min=0.0,
    sigma_max=1.0,
)

In [174]:
t_embeddings = film(batch.time)

In [175]:
t_embeddings.shape

torch.Size([3, 4, 14, 64])

In [177]:
# one layer's, all atoms, first l1 feature, first channel
t_embeddings[0,:,1:4,0]

tensor([[0.0291, 0.0291, 0.0291],
        [0.0291, 0.0291, 0.0291],
        [0.0291, 0.0291, 0.0291],
        [0.0291, 0.0291, 0.0291]], grad_fn=<SelectBackward0>)

All the same yay!

In [178]:
t_embeddings[0,:,1:4,1]

tensor([[-0.0120, -0.0120, -0.0120],
        [-0.0120, -0.0120, -0.0120],
        [-0.0120, -0.0120, -0.0120],
        [-0.0120, -0.0120, -0.0120]], grad_fn=<SelectBackward0>)

In [179]:
# different channel witha. different weight!

In [180]:
# different layer
t_embeddings[1,:,1:4,0]

tensor([[-0.0007, -0.0007, -0.0007],
        [-0.0007, -0.0007, -0.0007],
        [-0.0007, -0.0007, -0.0007],
        [-0.0007, -0.0007, -0.0007]], grad_fn=<SelectBackward0>)

In [181]:
# seems like its all working!